## Required Permissions:
## Compute cluster has "Data Scientist" access to the workspace 
## "Storage Blob Contributor" on the storage 

In [3]:
# Python
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input, load_component, command
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import JobService

In [4]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [5]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

print(ml_client)

Found the config file in: /config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fc959ce6f20>,
         subscription_id=6025ba02-1dfd-407f-b358-88f811c7c7aa,
         resource_group_name=sc1-ml1,
         workspace_name=sc1ml1)


In [6]:
parent_dir = "."
download_model = load_component(source=parent_dir + "/model_download.yaml")


In [55]:
# Construct pipeline
@pipeline()
def pipeline_with_components_from_yaml(
registry: str = "azureml",
model_name: str = 'model'

):
    """E2E dummy train-score-eval pipeline with components defined via yaml."""
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    download_model(
        registry=registry,
        model_name=model_name
    )

pipeline_job = pipeline_with_components_from_yaml(
    registry='azureml-meta',
    model_name='Llama-2-7b'
    
)

# set pipeline level compute
pipeline_job.settings.default_compute = "model-download-cluster1"

In [56]:
# submit the command
returned_job = ml_client.create_or_update(pipeline_job)

# view interactive service endpoints -- wait for job to be running before calling the below function
# ml_client.jobs.show_services(returned_job.name, 0)

In [7]:
package_model = load_component(source=parent_dir + "/package_model.yaml")


In [16]:
# Construct pipeline
@pipeline()
def pipeline_with_components_from_yaml(
registry: str = "azureml",
model_name: str = 'model',
model_version: str = 'latest'


):
    """E2E dummy train-score-eval pipeline with components defined via yaml."""
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    package_model(
        model_name=model_name,
        model_version=model_version, 
        model_download="path"

    )

pipeline_job = pipeline_with_components_from_yaml(
    registry='azureml-meta',
    model_name='Llama-2-7b',
    model_version='latest'
)

# set pipeline level compute
pipeline_job.settings.default_compute = "model-download-cluster1"

In [17]:
returned_job = ml_client.create_or_update(pipeline_job)


Exception: 
[37m
[30m
1) One or more files or folders do not exist.
[39m[39m

Details: 

[31m(x) No such file or directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/jacwang3/code/Users/jacwang/model_import_pipeline/path[39m

Resolutions: 
1) Double-check the directory path you provided and enter the correct path.
If using the CLI, you can also check the full log in debug mode for more details by adding --debug to the end of your command

Additional Resources: The easiest way to author a yaml specification file is using IntelliSense and auto-completion Azure ML VS code extension provides: [36mhttps://code.visualstudio.com/docs/datascience/azure-machine-learning.[39m To set up VS Code, visit [36mhttps://docs.microsoft.com/azure/machine-learning/how-to-setup-vs-code[39m
